In [2]:
import rawpy
import imageio
import numpy as np
from PIL import Image
from spectral import *
import spectral.io.aviris as aviris
import matplotlib
matplotlib.use('WXAgg')
import matplotlib.pyplot as plt
import wx
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from scipy.signal import argrelextrema
import scipy
import cv2
from sklearn import svm
import pickle
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import random
import cv2
spectral.settings.WX_GL_DEPTH_SIZE = 16

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from keras.initializers import random_uniform
import tensorflow as tf


tf.config.run_functions_eagerly(True)
w = 2048
band = 98

lambda_list = [
 601.00,
 602.00,
 606.00,
 610.00,
 612.36,
 616.73,
 621.20,
 624.07,
 626.71,
 628.05,
 631.63,
 636.59,
 641.24,
 646.58,
 651.24,
 654.85,
 658.63,
 660.78,
 665.41,
 669.57,
 673.58,
 683.94,
 688.45,
 692.56,
 696.90,
 699.05,
 703.90,
 708.03,
 712.41,
 719.38,
 724.03,
 727.76,
 731.65,
 734.10,
 737.94,
 741.27,
 744.62,
 752.29,
 758.12,
 762.98,
 767.84,
 770.30,
 775.42,
 780.03,
 784.55,
 790.34,
 795.45,
 799.87,
 804.17,
 806.41,
 810.84,
 814.31,
 817.33,
 827.88,
 832.47,
 836.32,
 840.31,
 842.50,
 846.97,
 850.85,
 854.61,
 859.38,
 863.32,
 866.84,
 870.24,
 872.16,
 875.86,
 879.01,
 881.74,
 886.01,
 889.95,
 893.54,
 897.35,
 899.58,
 903.55,
 906.80,
 909.86,
 914.24,
 917.38,
 920.24,
 923.17,
 924.90,
 928.03,
 930.66,
 933.14,
 942.38,
 945.32,
 947.88,
 950.70,
 952.39,
 955.99,
 959.30,
 962.35,
 966.54,
 969.13,
 971.34,
 973.58,
 975.08,
]


dic_wave_length = {}
for k in range(band):
    dic_wave_length[k] = lambda_list[k] 

In [ ]:


# for key, value in dic_wave_length.items():
#     if value<701 and value >699:
#         print((key, value))
def create_raw_cube(path, w=w, band=band):
    data_before_process = np.fromfile(path, dtype=np.uint16)
    # data_before_process = np.fromfile(path)
    h = int(len(data_before_process)/(w*band))

    data_1 = np.reshape(data_before_process, (h, w*band))
    
    cube = np.zeros((h, w, band), dtype = np.uint16)
    for k in range(h):
        cube[k,:,:] = np.transpose(np.reshape(data_1[k,:], (band, w)))
    return cube

def background_removal(cube_of_interest):
    h = int((np.shape(cube_of_interest)[0]))
    cropped_cube = np.zeros((h,w,band))
    vi = ndvi(cube_of_interest, 25, 49)
    vi_mask = np.where(vi < 0.3, 0, 1)
    for k in range(band):
        cropped_cube[:,:,k] = np.multiply(cube_of_interest[:,:,k],vi_mask)
    return cropped_cube

def find_interesting_band_with_successive_var(cropped_cube_of_interest): 
    var_list = []
    for k in range(band):
        var_list.append(np.var(cropped_cube_of_interest[:,:,k]))
    dvar = np.gradient(var_list, 1)
    print(argrelextrema(dvar, np.greater))
    return dvar

def create_smooth_cube(cube_of_interest):
    dim = np.shape(cube_of_interest)
    smoothed_cube = np.zeros((dim[0], dim[1], dim[2]), dtype=np.uint8)
    # for i in range(dim[0]):
    #     for j in range(dim[1]): 
    #         smoothed_cube[i,j,:] = scipy.signal.savgol_filter(cropped_cropped_cube[i,j,:], deriv = 1, polyorder = 3, window_length = 29)
    
    smoothed_cube[:,:,:] = scipy.signal.savgol_filter(cube_of_interest[:,:,:], axis = 2, deriv = 0, polyorder = 3, window_length= 15)
    return smoothed_cube

def create_final_cube (path):
    cube = create_raw_cube(path)
    cropped_cube = background_removal(cube)[:,250:1750,:]
    # smoothed_cube = create_smooth_cube(cropped_cube)
    # return smoothed_cube
    return cropped_cube

def signaltonoise(a, axis=0, ddof=0):
    vect = np.matrix.flatten(a)
    m = np.mean(vect)
    sd = np.std(vect)
    return np.where(sd == 0, 0, m/sd)

def denoise_image(cube_of_interest):
    dim = np.shape(cube_of_interest)
    print(dim)
    denoised_image = np.zeros((dim[0], dim[1], dim[2]), dtype='uint8')
    for k in range(dim[2]):
        slice = cube_of_interest[:,:,k]
        index = np.nonzero(slice)
        created_slice = np.zeros((dim[0], dim[1]))
        created_slice[index] = denoise_wavelet(slice[index], channel_axis = None,
                            method='BayesShrink', mode='soft',
                            rescale_sigma=True)
        # sliceCopy = np.uint8(slice)
        # for k in range(len(index[0])):
        #     created_slice[index[0][k]][index[1][k]] = cv2.fastNlMeansDenoising(sliceCopy[index])
        denoised_image[:,:,k] = created_slice
    return denoised_image

def create_LLSI_and_CCTR1_images(cube_of_interest):
    LLSI = np.divide(cube_of_interest[:,:,29]-cube_of_interest[:,:,0], cube_of_interest[:,:,29]+cube_of_interest[:,:,0]) - cube_of_interest[:,:,54]
    # CTR1 = np.divide(cube_of_interest[:,:,25], cube_of_interest[:,:,0])

    return LLSI
    # arr8 = ((LLSI-np.mean(LLSI))/np.var(LLSI)*256).astype(np.uint8)




In [4]:
cube_YR_1 = create_final_cube('high_leaf_1.raw')

In [9]:
cube_YR_2 = create_final_cube('high_leaf_2.raw')
cube_YR_3 = create_final_cube('high_leaf_3.raw')

In [ ]:
LLSI_1, CCTR1_1 = create_LLSI_and_CCTR1_images(cube_YR_1)

LLSI_2, CCTR1_2 = create_LLSI_and_CCTR1_images(cube_YR_2)

LLSI_3, CCTR1_3 = create_LLSI_and_CCTR1_images(cube_YR_3)

for k in [LLSI_1, LLSI_2, LLSI_3]:
    plt.figure()
    
    im = plt.imshow(k, cmap = 'jet')
    plt.colorbar(im)
    plt.show()

In [8]:
print(np.max(cube_YR_1))
print(np.max(cube_YR_2))
print(np.max(cube_YR_3))

415.0
397.0
386.0


In [28]:
index = np.nonzero(cube_YR_1[:,:,0])
cube_non_zero_YR_1 = cube_YR_1[np.array(index[0]), np.array(index[1]),:]

(2, 348254)


In [25]:
def view_heat_map(band):
    # index = np.nonzero(cube_YR_1[:,:,0])
    # print(np.shape(index))
    # cube_non_zero_YR_1 = np.array(cube_YR_1)[index[0], index[1],:]
    
    # arr8_1 = cube_YR_1[:,:,band].astype(np.uint8)
    plt.figure()
    vmax = max(np.max(cube_YR_1[:,:,band]), np.max(cube_YR_2[:,:,band]), np.max(cube_YR_3[:,:,band]))
    arr8_1 = cube_YR_1[:,:,band]
    im = plt.imshow(arr8_1, cmap = 'jet', vmax = vmax)
    plt.colorbar(im)
    plt.show()

    plt.figure()
    arr8_2 = cube_YR_2[:,:,band]     
    im2 = plt.imshow(arr8_2, cmap = 'jet', vmax = vmax)
    plt.colorbar(im2)
    plt.show()

    plt.figure()
    arr8_3 = cube_YR_3[:,:,band]
    # heatmap_3 = cv2.applyColorMap(arr8_3, cv2.COLORMAP_JET_R)                
    im3 = plt.imshow(arr8_3, cmap = 'jet', vmax = vmax)
    plt.colorbar(im3)
    plt.show()


In [174]:
view_heat_map(37)
view_heat_map(38)

In [173]:
print(lambda_list[34])

737.94
